In [1]:
import requests
import pandas as pd
import json
from parsel import Selector
import re
import time
import datetime
from tqdm import tqdm
pd.set_option('display.max_columns', None)

要获取的： 上次开播
订阅数：
公告：
工会

a：
    是否开启摄像头

b
贡献榜：是否超过50人 ，总多少
粉丝榜：

In [3]:
# 标签说明  
#- gameFullName:     游戏名称
# - gameHostName:     
# - boxDatainfo:     
# - totalCount:       人气值
# - roomName:         直播间名
# - bussType:       
# - screenshot:       直播截图
# - privateHost:      初始id？
# - nick:             昵称
# - avatar180:        头像
# - gid:          
# - introduction:     简介
# - recommendStatus:  推荐类型ID
# - recommendTagName: 推荐类型名称
# - isBluRay:         是否蓝光
# - bluRayMBitRate:   蓝光清晰度
# - screenType:       截屏类型
# - liveSourceType:   分区？？（一起玩、女神、云顶之弈...）
# - uid:              用户ID
# - channel:          工会id
# - livechannel:      工会id
# - imgRecInfo:
# - aliveNum:         
# - attribute:         使用英雄、是否上电视？
# - profileRoom:       房间号,可直接用于网址搜索房间
# - isRoomPay:         是否为付费房间
# - roomPayTag:        
# - isWatchTogetherVip: 是否为一起看vip房间
  
  
  
# - 主播动态视频
# - 公告栏
# - 周榜信息
# - 等级
# - 个人信息
#     - 订阅
#     - 性别
#     - 

In [4]:
ht = pd.read_csv('4月4日数据_已提取标签.csv')  # 主播信息表     ht   The host tabel
pht = pd.read_csv('主播个人信息.csv')          # 主播个人信息表 pht   personal host tabel
pl = pd.read_csv('房间信息.csv')
wc = pd.read_csv('周贡献榜.csv')   # 周贡献榜

###### 获取所在公会

In [159]:
# 获取uid对应的工会信息，列表channeList 
channel_None = {'isPlaintum': None, 'channelNumber': None, 'name': None, 'logo': None, 'isDiamond': None, 'channelId': None}
def get_channel(uid):
    url = f'https://chgate.huya.com/proxy/index?service=thrift_sign&iface=getSignChannelInfo&callback=getSignChannelInfo&data={uid}'
    r = requests.get(url)
    jsonData = json.loads(r.text[19:-1])
    if jsonData['data']:
        jsonData['data']['用户id'] = uid
        channelList.append(jsonData['data'])
    else:
        channel_None['用户id'] = uid
        channelList.append(channel_None)

channeList = []
a = pht['用户id'].apply(get_channel)   # a是为了不输出

In [246]:
# 一些数据上的处理，因为刚刚爬少写了channel_None的uid
# c = a[~a.用户id.isna()]
# drop_index = c[c.duplicated()].sort_values('用户id').index
# a.drop(drop_index, inplace=True)

# a['用户id'] = pht['用户id'].values

In [261]:
# 合并公会信息
# a = pd.DataFrame(channelList)
# a.to_csv('用户id-公会数据.csv', index=False)  # 保存公会数据
# pht = pd.merge(pht, a, on='用户id', how='left')
# pht.head()

,用户id,房间号,公会id,昵称,标签,isPlaintum,channelNumber,name,logo,isDiamond,channelId
0,1820796294,12214,1820796294,时代-十四剑姬,大神推荐,True,90214.0,时代游戏,https://huyaimg.msstatic.com/cdnimage/channell...,True,14732.0
1,2367547387,333003,2367547387,Zz1tai姿态,大神推荐,True,90999.0,皇族Rstar,https://huyaimg.msstatic.com/cdnimage/channell...,False,67547117.0
2,1346609715,660000,1346609715,英雄联盟赛事,魅力新星,True,66.0,虎牙游戏,https://huyaimg.msstatic.com/cdnimage/channell...,False,78941969.0
3,2183525275,572329,2183525275,Dae-北枫c,大神推荐,True,90725.0,招到期主播,https://huyaimg.msstatic.com/cdnimage/channell...,True,16634988.0
4,2334560814,420342,2334560814,时代-老枪赵信【李弟】,NaN,True,90214.0,时代游戏,https://huyaimg.msstatic.com/cdnimage/channell...,True,14732.0


In [284]:
# pht = pht.rename(columns={'channelNumber':'公会号', 'name':'公会名称', 'isDiamond':'是否为钻石会员', 'channelId':'公会id'})
# # pht.drop(columns=['logo', '公会id'], inplace=True)
# pht.to_csv('4月4日数据_已提取标签.csv', index=False)

In [285]:
pht

,用户id,房间号,昵称,标签,isPlaintum,公会号,公会名称,是否为钻石会员,公会id
0,1820796294,12214,时代-十四剑姬,大神推荐,True,90214.0,时代游戏,True,14732.0
1,2367547387,333003,Zz1tai姿态,大神推荐,True,90999.0,皇族Rstar,False,67547117.0
2,1346609715,660000,英雄联盟赛事,魅力新星,True,66.0,虎牙游戏,False,78941969.0
3,2183525275,572329,Dae-北枫c,大神推荐,True,90725.0,招到期主播,True,16634988.0
4,2334560814,420342,时代-老枪赵信【李弟】,NaN,True,90214.0,时代游戏,True,14732.0
...,...,...,...,...,...,...,...,...,...
11140,2323517119,16531038,伊芙-蕾雅,NaN,True,11720.0,【傲辰集团】,True,11720.0
11141,1279532989136,24503499,祈雪咒,NaN,None,NaN,None,None,NaN
11142,2214316071,23382664,、青栀、,NaN,True,64113.0,九九娱乐,False,64113.0
11143,1746647510,21582991,神人,NaN,None,NaN,None,None,NaN


###### 个人信息

In [337]:
pl = pd.DataFrame()
profileRoomList = []     # 用来记录不能获取个人信息的房间id
for n, profileRoom in enumerate(pht.房间号):
    url = f'https://www.huya.com/{profileRoom}'
    r = requests.get(url)
    selector = Selector(text=r.text)
    try:
        datas = selector.xpath('/html/body/script[7]').getall()[0]
        start_time = re.findall('var TT_META_DATA = {"start_time":(.*?)}', datas)[0]       # 时间戳
        room_data = json.loads(re.findall('var TT_ROOM_DATA = (.*?})', datas)[0])         # 房间信息
        profile_info = json.loads(re.findall('var TT_PROFILE_INFO = (.*?})', datas)[0])   # 主播个人信息
        merge_dic = {**profile_info, **room_data}     # 合并字典
        pl = pl.append(merge_dic, ignore_index=True)
    except:
        profileRoomList.append(profileRoom)           # 违规整改的
        print(url)
    print("\r| " + "█"*int(n/446) + f" | 【{format(n/11145, '.2%')}】", end='', flush=True)

|  | 【0.19%】https://www.huya.com/24493416
|  | 【0.28%】https://www.huya.com/24467478
|  | 【0.53%】https://www.huya.com/24342508
|  | 【1.39%】https://www.huya.com/24357894
|  | 【2.29%】https://www.huya.com/24462769
|  | 【2.36%】https://www.huya.com/24467123
|  | 【3.30%】https://www.huya.com/22344614
| █ | 【4.24%】https://www.huya.com/19393396
| █ | 【5.37%】https://www.huya.com/20807565
| █ | 【6.19%】https://www.huya.com/19064631
| █ | 【7.07%】https://www.huya.com/24458798
| ██ | 【9.32%】https://www.huya.com/23677137
| ██ | 【9.74%】https://www.huya.com/24477568
| ███ | 【12.44%】https://www.huya.com/13990934
| ████ | 【17.81%】https://www.huya.com/21640412
| ████ | 【19.65%】https://www.huya.com/24471118
| █████ | 【22.86%】https://www.huya.com/24488707
| ██████ | 【24.86%】https://www.huya.com/19784046
| ██████ | 【26.48%】https://www.huya.com/21982907
| ███████ | 【30.20%】https://www.huya.com/24497476
| ███████ | 【31.72%】https://www.huya.com/24483038
| ███████ | 【31.90%】https://www.huya.com/24108539
| ███████ 

In [356]:
# pht.drop(index=pht[pht.房间号.isin(profileRoomList)].index, inplace=True)      # 去掉违规整改的主播

In [ ]:
# pl.to_csv('房间信息.csv', index=False)
# pl = pl[['sex', 'bussType', 'cameraOpen', 'fans', 'isOn', 'gameFullName', 'isReplay', 'lp', 'recommendStatus', 'startTime', 'type']]
# pl = pl.rename(columns={'sex': '性别', 'bussType' : '直播类型', 'cameraOpen': '是否开启摄像头', 'fans': '粉丝数' , 'isOn': '是否在播中', 'gameFullName': '游戏类型', 'isReplay': '是否重播', 'lp': '用户id', 'recommendStatus': '推荐标签', 'startTime':'上次直播时间', 'type': '分类'})
# pl['上次直播时间'] = pl.上次直播时间.map(time.localtime).map(lambda x: time.strftime("%Y--%m--%d %H:%M:%S", x)).astype('datetime64')
# pht = pht.reset_index(); pht.drop(columns='index', inplace=True) # 重新索引

In [443]:
# 合并为个人信息
# pht = pd.merge(pht, pl, on='用户id', how='left')

###### 爬取公告

In [6]:
def get_announcement(profileRoom):
    url = f'https://www.huya.com/{profileRoom}'
    r = requests.get(url)
    selector = Selector(text=r.text)
    try:
        return selector.xpath('//*[@id="J_profileNoticeText"]/span/text()').getall()[0]
    except:
        print(url)
        return None
pht['公告'] = pht.房间号.apply(get_announcement)
pht.to_csv('个人信息.csv', index=False)

https://www.huya.com/660000
https://www.huya.com/868067
https://www.huya.com/660001
https://www.huya.com/24493416
https://www.huya.com/24467478
https://www.huya.com/24357894
https://www.huya.com/24462769
https://www.huya.com/24467123
https://www.huya.com/660113
https://www.huya.com/18525045
https://www.huya.com/20807565
https://www.huya.com/11948626
https://www.huya.com/19064631
https://www.huya.com/24477568
https://www.huya.com/13990934
https://www.huya.com/22402642
https://www.huya.com/19520860
https://www.huya.com/24471118
https://www.huya.com/660137
https://www.huya.com/24488707
https://www.huya.com/21982907
https://www.huya.com/24497476
https://www.huya.com/23188911
https://www.huya.com/24483038
https://www.huya.com/24108539
https://www.huya.com/24471704
https://www.huya.com/24179291
https://www.huya.com/20530064
https://www.huya.com/24488494
https://www.huya.com/24489024
https://www.huya.com/24489030
https://www.huya.com/24482904
https://www.huya.com/24179228
https://www.huya.com

In [12]:
pht

,用户id,房间号,公会id,昵称,标签,性别,直播类型,是否开启摄像头,粉丝数,是否在播中,游戏类型,是否重播,推荐标签,上次直播时间,分类,公告
62,1149399980,215609,1149399980,李阿特,魅力新星,0.0,1.0,0.0,662438.0,0.0,英雄联盟,0.0,496.0,1.617541e+09,NORMAL,直播时间 12:00-16:00 21:00-2:00 商务+Q：56155555 添...


###### 爬取视频动态

In [ ]:
# https://liveapi.huya.com/moment/getMomentListByUidForWeb?callback=jQuery1113007995658170363851_1617714491431&pid=1149399980&uid=0&seed=0&_=1617714491434
vi = pd.DataFrame()

for uid in tqdm(pht.用户id):
    url = f'https://liveapi.huya.com/moment/getMomentListByUidForWeb?callback=jQuery1113007995658170363851_1617714491431&pid={uid}&uid=0&seed=0&_=1617714491434'
    r = requests.get(url)
    video_info = json.loads(r.text[43:-1])['data']['moments']
    try:
        vi = vi.append(video_info, ignore_index=True)
    except:
        pass

In [13]:
vi = pd.read_csv('视频动态.csv')

,用户id,房间号,公会id,昵称,标签,性别,直播类型,是否开启摄像头,粉丝数,是否在播中,游戏类型,是否重播,推荐标签,上次直播时间,分类,公告
6531,17303541,351740,17303541,歌y,NaN,2.0,1.0,0.0,3800.0,1.0,英雄联盟,0.0,0.0,1.617606e+09,NORMAL,新主播求订阅～ 下午直播到晚上 感谢关注～


###### 爬取周贡榜

In [116]:
# https://www.huya.com/cache5min.php?m=WeekRank&do=getItemsByPid&pid=1149399980
wc = pd.DataFrame()

for uid in tqdm(pht.用户id):
    url = f'https://www.huya.com/cache5min.php?m=WeekRank&do=getItemsByPid&pid={uid}'
    r = requests.get(url)
    week_contribute = r.json()['data']['vWeekRankItem']
    wc_len = len(wc)
    try:
        wc =  wc.append(week_contribute, ignore_index=True)
        wc.loc[wc_len-1:, '用户id'] = uid
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████| 11145/11145 [46:47<00:00,  3.97it/s]


In [119]:
wc

,lUid,sNickName,iScore,iGuardLevel,iNobleLevel,sLogo,iUserLevel,tNobleLevel,iSFFlag,用户id
0,1786502568,村长洗头用飘柔,1941400,0,2,https://huyaimg.msstatic.com/avatar/1033/6b/7a...,29,"{'iNobleLevel': 2, 'iAttrType': 0}",0,1.820796e+09
1,1199515156356,赵茗茗同学,1798200,0,5,https://huyaimg.msstatic.com/avatar/1031/93/d0...,15,"{'iNobleLevel': 5, 'iAttrType': 0}",0,1.820796e+09
2,1698832695,雨滴【小木兰】,657400,0,1,https://huyaimg.msstatic.com/avatar/1030/30/f1...,16,"{'iNobleLevel': 1, 'iAttrType': 0}",0,1.820796e+09
3,1356924630,无痕,503200,0,4,https://huyaimg.msstatic.com/avatar/1096/d6/cf...,19,"{'iNobleLevel': 4, 'iAttrType': 0}",0,1.820796e+09
4,1199573401591,小蘑菇QAQ,498000,0,1,https://huyaimg.msstatic.com/avatar/1075/5c/91...,2,"{'iNobleLevel': 1, 'iAttrType': 0}",0,1.820796e+09
...,...,...,...,...,...,...,...,...,...,...
45013,1464086656,听说接待姐姐想睡我,5000,0,0,http://huyaimg.msstatic.com/avatar/1060/f6/0b9...,10,"{'iNobleLevel': 0, 'iAttrType': 0}",0,2.323517e+09
45014,1632134593,似水流年，繁华了谁,3700,0,0,http://huyaimg.msstatic.com/avatar/1060/2a/572...,6,"{'iNobleLevel': 0, 'iAttrType': 0}",0,2.323517e+09
45015,1707464702,魔法少女蒙多,1000,0,0,https://huyaimg.msstatic.com/avatar/1075/26/e4...,5,"{'iNobleLevel': 0, 'iAttrType': 0}",0,2.214316e+09
45016,880783781,玙丶苏苏酱的绯闻男友,33000,0,1,https://huyaimg.msstatic.com/avatar/1006/95/fa...,25,"{'iNobleLevel': 1, 'iAttrType': 0}",0,2.214316e+09


###### 爬取主播所在地、个人等级、个性签名

In [4]:
# pht = pd.read_csv(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\个人信息.csv')

城市、省份、个性签名、个人等级

In [15]:
import numpy as np

In [32]:
city_info = pd.DataFrame()

In [17]:
# 省份映射到区域  
proDic = {'黑龙江':'东北', '吉林':'东北', '辽宁':'东北',

'上海':'华东', '江苏':'华东', '浙江':'华东', '安徽':'华东', '福建':'华东', '江西':'华东', '山东':'华东', '台湾':'华东',

'北京':'华北', '天津':'华北', '山西':'华北', '河北':'华北', '内蒙古':'华北',  

'河南':'华中', '湖北':'华中', '湖南':'华中',  

'广东':'华南', '广西':'华南', '海南':'华南', '香港':'华南', '澳门':'华南',  

'四川':'西南', '贵州':'西南', '云南':'西南', '重庆':'西南', '西藏':'西南',  

'陕西':'西北', '甘肃':'西北', '青海':'西北', '宁夏':'西北', '新疆':'西北',
'海外':'海外', np.nan:'未知'}

In [37]:
for uid in pht[pht.省份.map(proDic).isna()].用户id:
    url = f'https://q.huya.com/message/index.php?m=Message&do=getUserExtendInfo&uid={uid}&callback=jsonp_021539682112189662'
    r = requests.get(url)
    dic = json.loads(r.text[25:-2])['data']
    dic['uid'] = uid
    city_info = city_info.append(dic, ignore_index=True)

107524968
2450085830
347550312
303323372
2388285699
2374052354
1279529734282
1015645610
1570708927
1495460806
1199573451995
1030037565
1834012273
2277231098
1199558241439
1199560414273
1199513178041
1199524193232
592241638
94911822
2252636076
1259522948495
83149793
1576072743
1022868157
114262549
986050574
73453417
188258209
108894026
543795422
1704946509
1858099093
1279520994004
1199530673939
1459592890
1279525372411
2200520220
1740058409
1124745781
495712680
260290307
135874020
1352454575
1812339109
2316188108
106391116
2208644642
1259556253588
1431893932
369376926
362689823
102338472
2362748051
1655502494
1199526652788
1199553548170
1259517213255
1715302829
2298111093
1144822605
1145672980
903458764
1734704253
1279518041821
1584201529
1487821463
1199572365388
1727757254
1199534080132
1259524841524
1259524798885
1641010897
1451958260
1199527080692
59886479
899516667
975332933
1724537603
1512791977
141378883
1419256277
1055317444
1533535164
1662939631
321413676
801618774
139134273
119

In [42]:
city_info.drop_duplicates('uid', inplace=True)    # 去掉重复着

In [100]:
for uid in city_info.uid:
    try:
        pht.loc[pht[pht.用户id==uid].index,['城市','省份']] = city_info[city_info.uid==uid][['city','province']].values
    except:
        pass

In [101]:
city_info

,city,province,signature,uid,userLevel
0,澳门,特别行政区,新浪微博:上单主播黑店百地 20:00~2:00,107524968,26.0
1,香港,香港,当代陶渊明,-2147483648,5.0
2,台湾,台湾,谢谢有你们,347550312,14.0
3,香港,香港,“人来人往 勿失勿忘”,303323372,33.0
7,香港,香港,没钱 别圈,1015645610,12.0
...,...,...,...,...,...
387,台湾,台湾,你没听过他的故事，怎么了解他的为人🌴,1259517121283,5.0
388,银川,宁夏,接-陪玩-定位-上段-直接加我,1199564791671,3.0
392,海口,海南,,1259512472199,4.0
398,黄南,青海,,1259533003088,2.0


In [102]:
city_info.to_csv('省份_签名_等级_部分数据.csv', index=False)

In [77]:
pht = pd.read_csv(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\个人信息.csv')
pht = pht.join(city_info)

In [104]:
pht.to_csv(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\个人信息.csv', index=False)

###### 爬取主播等级、周排名、本周成长值

In [ ]:
# rank = pd.DataFrame()
# for uid in tqdm(pht.用户id):
#     url = f'https://q.huya.com/yy/?m=ProfileLevel&do=getPresenterGrowRank&callback=jQuery1830514955527579928_1618016899702&pid={uid}'
#     r = requests.get(url)
#     rank = rank.append(json.loads(r.text[40:-1])['data']['tGrowItem'], ignore_index=True)

已爬取完毕，直接合并即可

In [55]:
rank = pd.read_csv('周排名_等级_成长值.csv')
rank.head(2)

,iLevel,iLightUp,iLiveState,iRoomId,lPid,sAction,sFaceURL,sNickname,tGrowInfo,lWeeklyExp,lWeeklyIncExp,iRank
0,39.0,1.0,0.0,0.0,1820796294,NaN,https://huyaimg.msstatic.com/avatar/1091/31/56...,时代-十四剑姬,"{'lWeeklyExp': '360000', 'lWeeklyIncExp': '305...",360000,305161,848
1,35.0,0.0,0.0,0.0,2367547387,NaN,https://huyaimg.msstatic.com/avatar/1076/84/3e...,Zz1tai姿态,"{'lWeeklyExp': '155000', 'lWeeklyIncExp': '435...",155000,43563,5283


In [56]:
rank = rank.rename(columns={'lPid':'用户id','iLevel':'主播等级', 'iLightUp':'是否点亮徽章', 'lWeeklyIncExp':'本周成长值','iRank':'本周排名'})

In [81]:
a = rank[['用户id','主播等级','是否点亮徽章','本周成长值','本周排名']]
a.head(2)

,用户id,主播等级,是否点亮徽章,本周成长值,本周排名
0,1820796294,39.0,1.0,305161,848
1,2367547387,35.0,0.0,43563,5283


In [82]:
# pht = pd.read_csv(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\个人信息.csv')
pht = pd.merge(pht, a, on='用户id', how='left')

In [83]:
pht.head(2)

,用户id,昵称,标签,公告,公会级别,公会规模,上次直播时间,房间号,性别,直播类型,...,最高人气,平均小时人气,city,province,signature,userLevel,主播等级,是否点亮徽章,本周成长值,本周排名
0,1820796294,时代-十四剑姬,大神推荐,直播时间晚上9点播到凌晨4点,钻石公会,中,2021-04-05 21:03:49,12214,1.0,1.0,...,3180593,2560319,成都,四川,,6.0,39.0,1.0,305161,848
1,2367547387,Zz1tai姿态,大神推荐,003带你小游峡谷~每天25点准时开播~再帮小弟卡个牌子爱死你们了！,白金公会,小,2021-04-04 22:32:57,333003,2.0,1.0,...,3874036,2948839,,,,9.0,35.0,0.0,43563,5283


In [86]:
pht = pht.rename(columns={'city':'城市','province':'省份','signature':'签名','userLevel':'个人等级'})

In [87]:
pht.to_csv(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\个人信息.csv', index=False)

##### 爬取热门英雄、英雄位置

###### 热门英雄

In [212]:
# 网址、表单、请求头  
url = 'https://www.op.gg/statistics/ajax2/champion/'
json = {'type':'picked',
'league':'',
'period':'month',
'mapId':1,
'queue':'ranked'}

headers = {
    'Host': 'www.op.gg',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0',
'Accept': '*/*',
'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
'Accept-Encoding': 'gzip, deflate, br',
'Content-Type': 'application/json; charset=UTF-8',
'X-Requested-With': 'XMLHttpRequest',
'Content-Length': '53',
'Origin': 'https://www.op.gg',
'Connection': 'keep-alive',
'Referer': 'https://www.op.gg/statistics/champion/',
'Cookie': '_ga=GA1.1.632153092.1601615811; _ga_HKZFKE5JEL=GS1.1.1618149497.5.1.1618149703.0; wcs_bt=55c48ac9e22bec:1618149705; Hm_lvt_29884b6641f1b5709cc89a8ce5a99366=1618149502; _hist=%EA%B9%80%EC%B9%98%EB%A7%B9%24%EC%A1%B0%ED%86%A0%EB%AF%BC%24%EC%B1%84%ED%8C%85%EC%95%88%EC%B9%98%EB%8A%94%EC%86%8C%EB%82%98; __gads=ID=023d96cfc44a6faf-2281a6dcd0c300b2:T=1601616021:S=ALNI_MbomlUhbLVKMdk7Seg2SxRHWfELCQ; cto_bundle=wDULZ19KaENsTWFhUllJQ3Z5dThkRUJkZVA2TnNHY2JpZjdTTHFqdXdlZURUcXpYTmVTM3B0WThzckxEUzQ5WDJFanpxMFdBcGNHOUp5aFNLRzhieHh6c0hNUEpHbnpxeWxzQSUyQjJWbHd0dTQ3Zzg4TEglMkJoOW9PaGxIRk1WblA4JTJCNEVzMVlsVGN2T2FSUkd0TU5Ua3ZCY0N5WVElM0QlM0Q; _dd_s=rum=0&expire=1618150604554; Hm_lpvt_29884b6641f1b5709cc89a8ce5a99366=1618149705; _gid=GA1.2.799906881.1618149504; _gat_gtag_UA_37377845_1=1; _gat_gtag_UA_140073778_1=1',
'Pragma': 'no-cache',
'Cache-Control': 'no-cache',
'TE': 'Trailers',
}

In [215]:
r = requests.post(url, json=json, headers=headers)

In [236]:
selector = Selector(r.text)

leauge = selector.xpath('//table//tr/td[3]/a/text()').getall()
leauge

['虚空之女',
 '探险家',
 '盲僧',
 '暴走萝莉',
 '暗夜猎手',
 '魂锁典狱长',
 '解脱者',
 '涤魂圣枪',
 '沙漠玫瑰',
 '麦林炮手',
 '战争之影',
 '破败之王',
 '疾风剑豪',
 '圣枪游侠',
 '熔岩巨兽',
 '封魔剑魂',
 '刀锋舞者',
 '皮城女警',
 '光辉女郎',
 '离群之刺',
 '曙光女神',
 '影流之主',
 '腕豪',
 '蒸汽机器人',
 '堕落天使',
 '不屈之枪',
 '魔法猫咪',
 '荒漠屠夫',
 '法外狂徒',
 '恶魔小丑',
 '狂野女猎手',
 '牛头酋长',
 '仙灵女巫',
 '战争女神',
 '远古巫灵',
 '蜘蛛女皇',
 '正义巨像',
 '时间刺客',
 '不灭狂雷',
 '深海泰坦',
 '寒冰射手',
 '无极剑圣',
 '德玛西亚之力',
 '刀锋之影',
 '德邦总管',
 '永恒梦魇',
 '不祥之刃',
 '暗裔剑魔',
 '迷失之牙',
 '戏命师',
 '血港鬼影',
 '诺克萨斯之手',
 '无双剑姬',
 '卡牌大师',
 '星籁歌姬',
 '影流之镰',
 '兽灵行者',
 '赏金猎人',
 '祖安狂人',
 '扭曲树精',
 '圣锤之毅',
 '九尾妖狐',
 '武器大师',
 '山隐之焰',
 '诡术妖姬',
 '未来守护者',
 '酒桶',
 '迅捷斥候',
 '含羞蓓蕾',
 '沙漠死神',
 '祖安怒兽',
 '暮光之眼',
 '放逐之刃',
 '诺克萨斯统领',
 '潮汐海灵',
 '雪原双子',
 '永猎双子',
 '复仇之矛',
 '青钢影',
 '铁铠冥魂',
 '机械先驱',
 '披甲龙龟',
 '符文法师',
 '虚空掠夺者',
 '天启者',
 '元素女皇',
 '惩戒之箭',
 '生化魔人',
 '冰晶凤凰',
 '发条魔灵',
 '亡灵战神',
 '残月之肃',
 '复仇焰魂',
 '暮光星灵',
 '痛苦之拥',
 '猩红收割者',
 '镕铁少女',
 '死亡颂唱者',
 '逆羽',
 '万花通灵',
 '众星之子',
 '远古恐惧',
 '齐天大圣',
 '牧魂人',
 '机械公敌',
 '德玛西亚皇子',
 '时光守护者',
 '皎月女神',
 '虚空恐惧',
 '邪恶小法师',
 '海洋之灾'

In [340]:
# 154个英雄 前30个为热门英雄。后30位冷门
legDic = {}
for n, key in enumerate(leauge):
    if n <= 50:
        legDic[key] = 1
    else:
        legDic[key] = 0

In [341]:
with open(r'D:\User\Document\Documents\狗熊会项目\狗熊会_美团商业分析\虎牙直播分析\datas\热门英雄字典.txt', 'w') as file:
    file.write(str(legDic))

###### 英雄位置

In [261]:
url = 'https://www.op.gg/champion/statistics'
headers = {'accept-language': 'zh-CN,zh;q=0.9'}
r = requests.get(url, headers=headers)

In [300]:
selector = Selector(r.text)
# /html/body/div[2]/div[2]/div[2]/div[1]/div[5]/div[3]/div[1]/a/div[1]
legRank = selector.xpath('//a/div/text()').getall()

In [333]:
# 以第一个位置为准
legRankDic = {}
for key,value in zip(legRank[153:-1],legRank[154:]):
    if len(key)<10:
        legRankDic[key] = value.split()[0]

In [335]:
# 保存数据
with open('英雄位置.txt', 'w') as f:
    f.write(str(legRankDic))

#### 每隔半小时爬取一次，爬取24小时的英雄联盟专区在播直播间信息

In [29]:
# ht44 = pd.read_csv('4月4日数据.csv')

In [21]:
def get_live_pages(df, conti=True):
    '''
    爬取当前英雄联盟专区直播页面的在播主播人气等信息
    df: 一个用于存放信息的dataframe（可以是空的dataframe）
    conti：是否每半小时爬取一次，False：只爬取当前信息这一次
    '''
    try:
        while True:
            total_page = 5; page = 1
            while page <= total_page:
                url = f'https://www.huya.com/cache.php?m=LiveList&do=getLiveListByPage&gameId=1&tagAll=0&callback=getLiveListJsonpCallback&page={page}'
                r = requests.get(url)
                b = json.loads(r.text[25:-1])
                get_time = b['data']['time']
                a = b['data']['datas']
                try:
                    df = df.append(a, ignore_index=True)
                    df['爬取时间'] = get_time
                    print('\r'+f'成功爬取第{page}页，共{total_page}页', end='')
                except:
                    print(f'爬取第{page}页失败')
                if total_page < b['data']['totalPage']:
                    total_page = b['data']['totalPage']
                else:
                    pass
                page += 1
            now = time.localtime(time.time())
            print('爬取时间：',[i for i in now[:-4]])

            # 爬取个人信息
            if conti==True:
                for i in range(120):     # 半小时后爬取
                    time.sleep(15) 
            else:
                return df
                break
    except:
        return df

# if __name__ == '__init__':
#     ht424 = pd.DataFrame()
#     ht424 = get_live_pages(ht424, conti=False)

In [75]:
a['totalCount'] = a.totalCount.astype(int)    # totalCount 转为数值格式 
hostSample15 = a[a.totalCount>300000][a.recommendTagName!='赛事精选'].sort_values('totalCount', ascending=False).head(15)

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [76]:
for rid in hostSample15.profileRoom:
    print(f'https://www.huya.com/{rid}')

https://www.huya.com/100
https://www.huya.com/518518
https://www.huya.com/230023
https://www.huya.com/941103
https://www.huya.com/641641
https://www.huya.com/660000
https://www.huya.com/912360
https://www.huya.com/575757
https://www.huya.com/526520
https://www.huya.com/972652
https://www.huya.com/981585
https://www.huya.com/891464
https://www.huya.com/24591072
https://www.huya.com/417019
https://www.huya.com/100953


In [2]:
ht422 = pd.read_csv('4月22日数据.csv')